In [6]:
from main import import_from_gsheet
merged_df = import_from_gsheet()

In [7]:
from bs4 import BeautifulSoup
import re

def get_all_lemma(df, lemma_col='詞條'):
    all_lemma = set(df[lemma_col].values)
    all_lemma2 = list(all_lemma)
    for l in list(all_lemma):
        if '*' in l:
            all_lemma2.remove(l)
    return all_lemma2

def index_lemma_in_def(df, all_lemma, def_col='釋義'):
    # Wrap <a> tag around lemma in definition
    indexed_defs = []
    pat = " (" + '|'.join(all_lemma) + ")[\.,;]? "
    for def_ in df[def_col].values:
        replacement = ' <a class="idx" href="#">\g<1></a> '
        def_ = re.sub(pat, replacement, def_)
        
        # Add href in <a> tag
        tag = BeautifulSoup(def_, "html.parser")
        for a in tag.find_all('a', attrs={'class':'idx'}):
            a["href"] = '#' + a.text.lower() + '_1'
        def_ = str(tag)
        indexed_defs.append(def_)
    return indexed_defs


all_lemma = get_all_lemma(merged_df)
merged_df['indexed_def'] = index_lemma_in_def(merged_df, all_lemma)
merged_df

,詞條,中文,音節,釋義,頁數,備註,轉寫者,indexed_def
0,Aûrras,一步,A-ûrras,"a step, a pace from 標示不清",6,,妤蓁_13-41,"a step, a pace from 標示不清"
1,Ailijt,粗草的莖,A-ilijt,the stalk or stem of coarse grass,6,,妤蓁_13-41,the stalk or stem of coarse grass
2,Aomo,女士的髮帶,A-omo,a lady's fillet for the hair,6,,妤蓁_13-41,a lady's fillet for the hair
3,Abak,小船,Abak,a small boat or sampan,6,,妤蓁_13-41,a small boat or sampan
4,Abûs,海,Abûs,the sea,6,,妤蓁_13-41,the sea
...,...,...,...,...,...,...,...,...
2896,Tsikarra,長尾巴的小田蜥蜴,Tsi_kar_ra,"smaller ditto, with a long tail",360,,峻維_326-360,"smaller ditto, with a long tail"
2897,Tsipollopollul,有斑點的房子蜥蜴,Tsi_pol_lo_pol_lul,a spotted house lizard,360,,峻維_326-360,a spotted house lizard
2898,Tsipan,西方,Tsi_pan,the west,360,,峻維_326-360,the west
2899,Tsiotsio,時鐘,Tsio_tsio,"a clock, or bell",360,,峻維_326-360,"a clock, or bell"


In [8]:
merged_df['indexed_def'].values

array(['a step, a pace from 標示不清', 'the stalk or stem of coarse grass',
       "a lady's fillet for the hair", ..., 'the west',
       'a clock, or bell',
       'fish. <a class="idx" href="#tsi_1">Tsi</a> ballai, all sorts of small fish, catfish. Tsisàbbi, carp. I\'si meried, eels. <a class="idx" href="#tsi_1">Tsi</a> maarrát, shellfish. <a class="idx" href="#tsi_1">Tsi</a> rodo, cod. <a class="idx" href="#tsi_1">Tsi</a> tarra boetsina, a roach: tsi tarra pagaoûbaas, a sword fish. <a class="idx" href="#tsi_1">Tsi</a> taiáûs, a sand-creeper. <a class="idx" href="#tsi_1">Tsi</a> rócot, a smelt. <a class="idx" href="#tsi_1">Tsi</a> teen, a fish, that has scales like the carp, but smaller. <a class="idx" href="#tsi_1">Tsi</a> kabea, almost like the teen, but smaller. <a class="idx" href="#tsi_1">Tsi</a> taddai, a sole. <a class="idx" href="#tsi_1">Tsi</a> taddadach, a Chinese fish, full of bones. <a class="idx" href="#tsi_1">Tsi</a> tillabis, a bream. Tai paûsosan, a broad fish with a sha